In [28]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"], port="9042")
session =   cluster.connect('creditcard')

## Lấy dữ liệu từ Cassandra

In [29]:

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
stmt1 = "SELECT * FROM non_fraud_transaction ;"
stmt2 = "SELECT * FROM fraud_transaction ;"
stmt3 = "SELECT * FROM customer ;"

In [31]:
data = session.execute(stmt1)
df_no = pd.DataFrame([ d for d in data])
     

In [32]:
df_no.shape

(401451, 11)

In [33]:
df_no = df_no.sample(n = 20000)

In [34]:
data = session.execute(stmt2)
df_yes = pd.DataFrame([ d for d in data])
     

In [35]:
df_yes.shape

(1499, 11)

In [36]:
data = session.execute(stmt3)
df_customer = pd.DataFrame([ d for d in data])
     

In [37]:
df_all = pd.DataFrame(np.vstack((df_yes, df_no)), columns=df_yes.columns)

In [38]:
df_all.head(4)

,cc_num,trans_time,age,amt,category,distance,is_fraud,merch_lat,merch_long,merchant,trans_num
0,6511349151405438,2019-12-27,73,780.81,shopping_pos,0.440991,1.0,39.605795,-115.254874,"fraud_Yost, Schamberger and Windler",1944259054c77cddcb9432bd9211ee68
1,6511349151405438,2019-12-26,73,355.27,grocery_pos,0.685475,1.0,39.80265,-115.02952,fraud_Rau and Sons,c85ac28478762467ef81204808666712
2,4208110975550360171,2020-03-15,38,1107.33,shopping_pos,1.249906,1.0,32.521879,-83.583698,fraud_Pouros-Conroy,fc1c0daa3e30873506750b76c4468a36
3,4208110975550360171,2020-03-14,38,356.49,grocery_pos,0.711085,1.0,31.320803,-83.104977,"fraud_Moen, Reinger and Murphy",e00bf51b76dcdccc4ac6804f6c6f94e2


In [39]:
df = pd.merge(df_all , df_customer, how='left' , left_on= 'cc_num' , right_on = 'cc_num')

In [40]:
df.head(4)

,cc_num,trans_time,age,amt,category,distance,is_fraud,merch_lat,merch_long,merchant,...,job,last,lat,long,max_amt,mean_amt,min_amt,state,street,zip
0,6511349151405438,2019-12-27,73,780.81,shopping_pos,0.440991,1.0,39.605795,-115.254874,"fraud_Yost, Schamberger and Windler",...,Interpreter,Nguyen,39.3426,-114.8859,2551.98,61.310324,1.01,NV,74835 Garner Point,89319
1,6511349151405438,2019-12-26,73,355.27,grocery_pos,0.685475,1.0,39.80265,-115.02952,fraud_Rau and Sons,...,Interpreter,Nguyen,39.3426,-114.8859,2551.98,61.310324,1.01,NV,74835 Garner Point,89319
2,4208110975550360171,2020-03-15,38,1107.33,shopping_pos,1.249906,1.0,32.521879,-83.583698,fraud_Pouros-Conroy,...,Audiological scientist,Bonilla,31.6143,-82.7243,1206.74,99.201684,26.70,GA,82073 Jessica Camp,31567
3,4208110975550360171,2020-03-14,38,356.49,grocery_pos,0.711085,1.0,31.320803,-83.104977,"fraud_Moen, Reinger and Murphy",...,Audiological scientist,Bonilla,31.6143,-82.7243,1206.74,99.201684,26.70,GA,82073 Jessica Camp,31567


In [41]:
X = df[['amt' , 'age' , 'distance' , 'max_amt',	'mean_amt'	,'min_amt']]

In [42]:
y = df['is_fraud'].astype('int')

## Chia tập dữ liệu: Train and Test

In [43]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.3, stratify=y, random_state=17)

## Lựa chọn Model 

In [17]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=2,max_depth=4)
dt.fit(X_train, y_train)

from sklearn.linear_model import LogisticRegression
lg=LogisticRegression()
lg.fit(X_train,y_train)

from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=17)
rf_clf.fit(X_train, y_train) 


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomForestClassifier(random_state=17)

In [18]:
dt_pre = dt.predict(X_test)
lg_pre = lg.predict(X_test)
rf_pre = rf_clf.predict(X_test)

In [19]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

mse_test = MSE(y_test,dt_pre)


rmse_test = mse_test**0.5


print('Test set RMSE of DecisionTree: {:.3f}'.format(rmse_test))
print("test acc : {}".format(accuracy_score(y_test,dt_pre)))
print("Test F1 : {}".format(f1_score(y_test,dt_pre)))

Test set RMSE of DecisionTree: 0.143
test acc : 0.9796899224806201
Test F1 : 0.8485549132947977


In [20]:
mse_test = MSE(y_test,lg_pre)
rmse_test = mse_test**0.5


print('Test set RMSE of LogisticRegression: {:.3f}'.format(rmse_test))
print("test acc : {}".format(accuracy_score(y_test,lg_pre)))
print("Test F1 : {}".format(f1_score(y_test,lg_pre)))

Test set RMSE of LogisticRegression: 0.159
test acc : 0.9747286821705426
Test F1 : 0.8080094228504122


In [21]:
from sklearn.metrics import mean_squared_error as MSE
mse_test = MSE(y_test,rf_pre)
rmse_test = mse_test**0.5


print('Test set RMSE of RandomForest: {:.3f}'.format(rmse_test))
print("test acc : {}".format(accuracy_score(y_test,rf_pre)))
print("Test F1 : {}".format(f1_score(y_test,rf_pre)))


Test set RMSE of RandomForest: 0.144
test acc : 0.9793798449612403
Test F1 : 0.846242774566474


### Lựa chọn Model có kết quả F1-score cao nhất
 => Model Random Forest

In [22]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold 
skf = StratifiedKFold(n_splits = 5)

params = {'max_features': [1, 3, 5] ,
          'max_depth' : [1, 3, 5, 7, 9],
          'random_state': [0, 5, 7, 10, 13, 15,17,20]}

rf =  RandomForestClassifier(random_state=17)
rf_grid = GridSearchCV(rf , params, cv=skf, scoring='accuracy')
rf_grid.fit(X_train, y_train) 

y_pred = rf_grid.predict(X_test)
scores = accuracy_score(y_pred, y_test)



In [23]:
rf_grid.best_params_, rf_grid.best_score_

({'max_depth': 7, 'max_features': 5, 'random_state': 13}, 0.9808624182822518)

In [24]:
from sklearn.ensemble import RandomForestClassifier
final = RandomForestClassifier(random_state=13, max_depth = 7,max_features=5 )
final.fit(X_train, y_train) 

RandomForestClassifier(max_depth=7, max_features=5, random_state=13)

In [25]:
from sklearn.metrics import  classification_report
print(classification_report(y_test, final.predict(X_test)))



              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6000
           1       0.89      0.82      0.85       450

    accuracy                           0.98      6450
   macro avg       0.94      0.90      0.92      6450
weighted avg       0.98      0.98      0.98      6450



In [26]:
pd.DataFrame({'feat': X.columns,
              'coef': np.abs(final.feature_importances_).flatten().tolist()}).\
                                        sort_values(by='coef', ascending=False).head()

,feat,coef
0,amt,0.904856
4,mean_amt,0.043964
2,distance,0.017374
5,min_amt,0.012403
3,max_amt,0.011160


## Lưu Model

In [27]:
dir = "D:/demolambda/flask"
import joblib
import os
joblib.dump(final , os.path.join(dir, 'model.pkl'))

['D:/demolambda/flask\\model.pkl']